In [13]:
!pip install markovify pytz numpy

In [80]:
from typing import Dict, List

import markovify

import prompts
import utils

# List of all unique prompts from July 5th to the morning of August 5th
dream_prompts: List[str] = utils.load_data_file("dream_prompts.pkl") 
    
# Markov models trained on the above
full_markov_model: markovify.Text = utils.load_data_file("markov.pkl")
    
# The reverse markov model predicts prompts backwards. 
# Making the forward and reverse models aboth create a sentences with a specific word allows joining them
# to make sentences that have that word in the middle. Otherwise one can only make sentences starting with the word
rev_full_markov_model: markovify.Text = utils.load_data_file("rev_markov.pkl")
    
# The dict's keys are arguments eg: "--width". The values are other dicts.
# The sub-dictionaries map the values it's parent argument may take to the number of times those values were used
arg_val_cnt: Dict[str, Dict[str, int]] = utils.load_data_file("arg_val_cnt.pkl")
arg_selector = prompts.ArgSelector(arg_val_cnt, len(dream_prompts))        

# A dictora
prompts_by_user: Dict[str, List[prompts.UserPrompt]] = utils.load_data_file("prompts_by_user.pkl")
    
username_to_models = {}

In [76]:
def get_markov_models(user):
    if user is None:
        return full_markov_model, rev_full_markov_model
    else:
        if user not in prompts_by_user:
            raise Exception(f"Not prompts exist for {user}")
        if user not in username_to_models:
            user_prompt_text = utils.remove_none_and_dups([user_prompt.prompt for user_prompt in prompts_by_user[user]])
            username_to_models[user] = markov.get_markov_models(user_prompt_text)
        return username_to_models[user]
    
def print_spaced(sents):
    print("\n\n".join(sents))

In [78]:
import markov
to_generate = 10 #@param {type:"int"}

# The max overlap with sentences in training prompts
max_overlap = 0.6 #@param {type:"float"}

# Set this value to make prompts based on a specific user
user = None #@param {type:"str"}

curr_markov_model, _ = get_markov_models(user)
prompts = markov.make_markov_sentences(to_generate, curr_markov_model, arg_selector, max_overlap=max_overlap)
print_spaced(prompts)

beautiful realistic photograph of a horrifying eldrich monster in a forest featured in national geographic, Asher Brown Durand, thomas cole, evangelion --height 512 --width 704 --steps 150 --cfg_scale 12 --sampler k_lms --seed 1337 --step 150

great white shark attack scarlett johnannson bleeding arm of scarlett johnannson featured in national geographic, Asher Brown Durand, thomas cole --height 576 --width 512 --steps 150 --cfg_scale 7 --sampler k_dpm_2_ancestral --seed 12345 --step 150

beautiful ultradetailed distant shot painting photograph of an incredible upper back watercolor dungeons and dragons featured in national geographic blood biting Thomas Cole, Asher Brown Durand, thomas cole --height 512 --width 512 --steps 100 --cfg_scale 7 --sampler k_euler_ancestral --seed 12345 --step 150

stranded exploration robot stuck on the ashy ocean floor near an island on a dead planet with dead fish, James Clyne, Joseph Cross, Asher Brown Durand, ArtStation --height 768 --width 1024 --step

In [74]:
# The keyword version creates a number of forward and reverse parts
# then mixes them. It will generate twice this number of prompts.
num_mix_part=10 #@param {type:"int"}

# The max overlap with sentences in training prompts
max_overlap = 0.6 #@param {type:"float"}

# Required word in the prompt
key_word = "photograph" #@param {type:"str"}

# Substitution for the above word
key_word_replacement = None #@param {type:"str"}

# Set this value to make prompts based on a specific user
user = None #@param {type:"str"}


curr_markov_model, curr_rev_markov_model = get_markov_models(user)
prompts = markov.make_markov_sentences_containing(to_generate, key_word,
                                               curr_markov_model, curr_rev_markov_model, arg_selector,
                                               key_word_replacement,
                                               max_overlap=max_overlap)
    
print_spaced(prompts)

a weird gross intelligent non-newtonian fluid bioluminescent alien creature from an extremely surrealistic DMT dimension by Jim Henson, quaint candy cottage, claymationphotograph from a center of an empty room with large windows, highly detailed, octane render, unreal engine, redshift, trending on artstation, terrifying atmosphere --height 1024 --width 768 --steps 150 --cfg_scale 10 --sampler k_dpm_2_ancestral --seed 2568911274 --step 150

Rick and Morty animated series is beautiful. a very beautiful warpfusion vector logo resembling fusion core in the shape of a kitty,photograph of hydrocephalic mark zuckerberg as a love interest in a romance anime --height 512 --width 768 --steps 150 --cfg_scale 11 --sampler k_dpm_2_ancestral --seed 204525742 --step 50 --h 1024

a tweet by donald trump, modern 2022photograph of a hyperrealistic Elmo --height 768 --width 512 --steps 70 --cfg_scale 6 --sampler ddim --seed 204525742 --step 150 --w 1920

a cartoon illustration of albert einstein as the s